Load CSV file into memory

In [1]:
import pandas as pd
import requests
import json
import time
from pandas.io.json import json_normalize
 
df = pd.read_csv('Global_country_populations_2013.csv') 
df = df[['CountryName','lat','lon']]
df.head()

,CountryName,lat,lon
0,Aruba,12.52368,-70.03487
1,Andorra,42.55920,1.57020
2,Afghanistan,34.53156,69.12535
3,Angola,-12.33290,16.83090
4,Albania,41.32233,19.82516


Define function to call LocationIQ API for reverse lookup of lat and long

In [2]:
def get_reverse_geocode_data(row, key):
    try: 
        API_KEY = key
        url = 'https://us1.locationiq.org/v1/reverse.php?key=' + API_KEY + '&lat=' + str(row['lat']) + '&lon=' + str(row['lon']) + '&format=json'
        
        response = (requests.get(url).text)
        #print(API_KEY)
        response_json = json.loads(response)
        time.sleep(0.5)
        return response_json
    
    except Exception as e:
        raise e

Call the function get_reverse_geocode_data using df columns as parameters

In [ ]:
with open("config/secret.json", "r") as read_file:
    data = json.load(read_file)
    key = data['key']
    
read_file.close()
        
df['API_response'] = df.apply(get_reverse_geocode_data, args=(key,), axis=1)
df['API_response'].head()
